# Convoluted Neural Network

In [1]:
''' Needed libraries '''

import numpy as np # For matrix operations and numerical processing
import matplotlib.pyplot as plt # For plotting
import os, sys # For filepaths
# Neural Network libraries:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
''' Add the datasets and libraries to the system path '''

# Find the path to our implementations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
home_directory = os.path.dirname(parent_directory)
libraries_path = os.path.join(home_directory, 'Libraries')

# Find the path to the datasets
datasets_path = os.path.join(home_directory, 'Datasets')

# Add them both to the system path
sys.path.append(datasets_path)
sys.path.append(libraries_path)

In [3]:
''' Loading in the training and test sets '''

training_set = np.load(os.path.join(datasets_path, 'fashion_train.npy'))  # Load training set
train_X = training_set[:, :-1] # Define X as all columns except the last one
train_y = training_set[:, -1] # Define y as the last column

test_set = np.load(os.path.join(datasets_path, 'fashion_test.npy'))  # Load test set
test_X = test_set[:, :-1] # Define X as all columns except the last one
test_y = test_set[:, -1] # Define y as the last column

# Checking sizes of datasets
print(train_X.shape) # 10000 pictures, 784 pixels each
print(train_y.shape) # 10000 labels
print(test_X.shape) # 5000 pictures, 784 pixels each
print(test_y.shape) # 5000 labels


print(train_X[0].shape) # Pictures are vectors with length 784
plot = train_X[0].reshape(28, 28) # Pictures need to be reshaped to 28x28 to be plotted (784 pixels total)
print(plot.shape) # Pictures are now 28x28

(10000, 784)
(10000,)
(5000, 784)
(5000,)
(784,)
(28, 28)


In [4]:
''' Resize the pixel values to have mean 0 and standard deviation 1 '''
# Calculate the mean and standard deviation of the training set
mean = np.mean(train_X)
std = np.std(train_X)

# Apply the transformation to both the training and test sets
train_X = (train_X - mean) / std
test_X = (test_X - mean) / std

In [5]:
''' Convert the data to tensors to be used by PyTorch '''
tensor_train_X = torch.from_numpy(train_X).float()
tensor_train_y = torch.from_numpy(train_y).long()
tensor_test_X = torch.from_numpy(test_X).float()
tensor_test_y = torch.from_numpy(test_y).long()

In [6]:
''' Batch size '''
batch_size = 100 # Number of samples in each batch

''' Reshape all items in both sets to be 28x28 instead of 784x1 '''
tensor_train_X = tensor_train_X.reshape(-1, 1, 28, 28)
tensor_test_X = tensor_test_X.reshape(-1, 1, 28, 28)

In [37]:
''' Loading data '''
train_loader = DataLoader(list(zip(tensor_train_X, tensor_train_y)), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(list(zip(tensor_test_X, tensor_test_y)), batch_size=batch_size, shuffle=False)

for X, y in train_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([100, 1, 28, 28])
Shape of y: torch.Size([100]) torch.int64


In [42]:
''' Creating models '''

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (stack): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=3136, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [22]:
''' Training loop '''

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
''' Test loop '''
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
''' Run CNN '''

def runCNN(epochs, loss_fn, optimizer):
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        test(test_loader, model, loss_fn)
    print("Done!")

In [43]:
''' Running the CNN '''

# Learning rate
learning_rate = 1e-3

# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer= torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
runCNN(epochs, loss_fn, optimizer)

Epoch 1
-------------------------------
loss: 2.279206  [  100/10000]
Test Error: 
 Accuracy: 79.4%, Avg loss: 0.528303 

Epoch 2
-------------------------------
loss: 0.563574  [  100/10000]
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.388808 

Epoch 3
-------------------------------
loss: 0.285393  [  100/10000]
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.384850 

Epoch 4
-------------------------------
loss: 0.327432  [  100/10000]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.351346 

Epoch 5
-------------------------------
loss: 0.276462  [  100/10000]
Test Error: 
 Accuracy: 86.4%, Avg loss: 0.371731 

Epoch 6
-------------------------------
loss: 0.321615  [  100/10000]
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.386662 

Epoch 7
-------------------------------
loss: 0.251892  [  100/10000]
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.349978 

Epoch 8
-------------------------------
loss: 0.130393  [  100/10000]
Test Error: 
 Accuracy: 88.2%, Avg loss: 0.332606 

Epoch 9
----------------

In [ ]:
# Optimize learning rate
learning_rate_list = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
for lr in learning_rate_list:
    sys.stdout = open(f"output.txt", "a")
    print(f"Learning rate: {lr}")
    runCNN(epochs, loss_fn, optimizer, lr=lr)
    sys.stdout.close()